## Word2Vec ## 

In this notebook we will go through the step by step creation of the Continouous Bag Of Words (CBOW).
CBOW is an embedded model that makes use of a "fake task" -> [within short window, predict the current word] to extract a vector that shows the relationship between words.

### Continuous Bag Of Words ###

Adapted from Robert Guthrie

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.linalg

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
# CBOW is a window view; we are trying to infer the word in the middle.
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right

raw_text= """Long Short-Term Memory (LSTM) is a recurrent neural network (RNN) architecture that has been designed
to address the vanishing and exploding gradient problems of conventional RNNs. Unlike feedforward neural networks,
RNNs have cyclic connections making them powerful for modeling sequences. 
They have been successfully used for sequence labeling and sequence prediction tasks,
such as handwriting recognition, language modeling, phonetic labeling of acoustic frames. However, in contrast to the deep neural
networks, the use of RNNs in speech recognition has been limited to phone recognition in small scale tasks. 
In this paper, we present novel LSTM based RNN architectures which make more effective
use of model parameters to train acoustic models for large vocabulary speech recognition. 
We train and compare LSTM, RNN and DNN models at various numbers of parameters and configurations.
We show that LSTM models converge quickly and give state of the art speech recognition performance for relatively small sized models.""".split()

# By deriving a set from "raw_text", we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

# Basic Tokenizer
word_to_ix = {word: i for i, word in enumerate(vocab)}

print(len(raw_text))
print(vocab_size)

152
106


In [4]:
# Now lets create a "dataset"
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = []
    for j in range(CONTEXT_SIZE, 0, -1):
        context.append(raw_text[i - j])

    for j in range(1, CONTEXT_SIZE + 1):
        context.append(raw_text[i + j])
        
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


[(['Long', 'Short-Term', '(LSTM)', 'is'], 'Memory'), (['Short-Term', 'Memory', 'is', 'a'], '(LSTM)'), (['Memory', '(LSTM)', 'a', 'recurrent'], 'is'), (['(LSTM)', 'is', 'recurrent', 'neural'], 'a'), (['is', 'a', 'neural', 'network'], 'recurrent')]


### Create the CBOW Model (as we have seen already other ANN) ###

We have to extend from nn.Module as all the other ANN

In [5]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embed_dim, context, hidden_size):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(context*embed_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, vocab_size),
            nn.LogSoftmax(dim = -1)
        )
        
    def forward(self, inputs):
#         print(inputs.shape)
#         print(inputs)
        out = self.embedding(inputs)
#         print(out.shape)
        out = out.view(1, -1)
#         print(out.shape)
        out = self.linear(out)
#         print(out.shape)
        return out
    
    # This is what we are actually interested on
    def get_word_vector(self, word):
        out = self.embedding(word)
        return out


#### Lets break it down! ####

In [6]:
VOCAB_SIZE = len(vocab)
EMBEDD_DIM = 10
BATCH_SIZE = 6
FULL_CONTEXT_SIZE = CONTEXT_SIZE * 2
HIDDEN_SIZE = 256

example_tensor = torch.randint(0, VOCAB_SIZE, [BATCH_SIZE, FULL_CONTEXT_SIZE])
print(example_tensor)
print(example_tensor.shape)

tensor([[  0,  60,  31,  54],
        [ 34,  16,   6,  60],
        [ 61,  26,  72, 101],
        [ 98,  87,  81,  52],
        [ 97,  50,  86,  47],
        [ 66,  75,   0,  72]])
torch.Size([6, 4])


In [7]:
CBOW_embedding = nn.Embedding(VOCAB_SIZE, EMBEDD_DIM)
example_result = CBOW_embedding(example_tensor)

# Now we have a representation of the words in a vector of EMBEDD_DIM Dimensions
print(example_result.shape) # [batch_size, context_size*2, EMBEDD_DIM]

# example_result = torch.flatten(example_result, start_dim=1)
example_result = example_result.view(BATCH_SIZE, -1)
print(example_result.shape)

torch.Size([6, 4, 10])
torch.Size([6, 40])


In [8]:
CBOW_hidden = nn.Linear(EMBEDD_DIM * FULL_CONTEXT_SIZE, HIDDEN_SIZE)
CBOW_hidden_relu = nn.ReLU()

example_result = CBOW_hidden(example_result)
example_result = CBOW_hidden_relu(example_result)
print(example_result.shape)

torch.Size([6, 256])


In [9]:
CBOW_output = nn.Linear(HIDDEN_SIZE, VOCAB_SIZE)
CBOW_output_soft = nn.LogSoftmax(dim = -1)

example_result = CBOW_output(example_result)
example_result = CBOW_output_soft(example_result)
print(example_result.shape) # 106 (vocab_size) -> [0.002, 0.003, 0.02, ... , n] see which one of the elements are the highest

torch.Size([6, 106])


In [10]:
print(example_result[0].argmax(-1))
print(example_result[0])
print(example_result[1].argmax(-1))
print(example_result[1])
# print(example_result[2].argmax(-1))
# print(example_result[3].argmax(-1))
# print(example_result[4].argmax(-1))

tensor(72)
tensor([-4.7508, -4.3116, -4.8388, -4.9742, -4.7032, -4.9129, -4.4857, -4.8750,
        -4.9746, -4.8454, -4.7147, -4.7026, -4.6719, -4.7485, -4.9168, -4.9041,
        -5.0626, -4.6345, -4.3543, -4.4781, -4.8308, -4.8335, -5.0236, -4.7701,
        -4.5072, -4.8972, -4.6358, -4.4334, -4.4260, -4.8470, -4.5422, -4.5713,
        -4.5250, -4.9929, -4.7054, -4.8444, -4.2976, -4.6372, -4.7281, -4.6088,
        -4.5065, -4.3125, -4.7630, -5.0298, -4.3554, -4.3312, -4.5655, -4.9850,
        -4.5924, -4.2130, -4.8103, -4.3189, -4.4505, -5.0468, -4.8018, -4.5481,
        -5.2290, -4.6892, -4.6284, -4.5324, -4.4145, -4.8371, -4.6775, -4.3782,
        -5.1181, -4.8188, -5.2095, -4.5977, -4.4670, -4.4855, -5.0328, -4.6065,
        -4.1777, -4.9149, -4.5627, -4.7859, -4.5433, -4.9432, -4.7474, -4.9997,
        -4.9390, -4.5285, -5.0326, -4.9508, -4.4573, -4.6170, -4.3230, -4.6923,
        -4.6697, -4.4527, -4.7838, -4.7805, -4.7263, -4.6637, -4.4790, -4.5909,
        -4.4835, -4.9858, -4.

## Back to the notebook ##

In [11]:
# Simple helper method to transform the context to the expected int vector - tensor

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix)

tensor([63, 89, 33, 60])

In [12]:
def train(model, epochs, data, optimizer, loss_fn):
    model.train()
    losses = []
    for epoch in range(epochs):
        total_loss = 0
        for context, target in data:

            # Prepare inputs and targets 
            context_idxs = make_context_vector(context, word_to_ix)
            context_idxs = context_idxs.to(device)
            
            target_id = make_context_vector([target], word_to_ix)
            target_id = target_id.to(device)

            # Do not accumulate 
            model.zero_grad()

            # Step 3. Run the forward pass
            log_probs = model(context_idxs)
    #         break

            # Step 4. Compute your loss function.
            loss = loss_fn(log_probs, target_id)

    #         loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

            # Step 5. Do the backward pass and update the gradient
            loss.backward()
            optimizer.step()

            # Get the Python number from a 1-element Tensor by calling tensor.item()
            total_loss += loss.item()
        losses.append(total_loss)
    return losses
    

In [13]:
VOCAB_SIZE = len(vocab)
EMBEDD_DIM = 10
BATCH_SIZE = 6
FULL_CONTEXT_SIZE = CONTEXT_SIZE * 2
HIDDEN_SIZE = 256

loss_function = nn.NLLLoss() # Because we are using Log_softmax
model = CBOW(vocab_size, EMBEDD_DIM, FULL_CONTEXT_SIZE, HIDDEN_SIZE)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

losses = train(model, 100, data, optimizer, loss_function)
model.eval()

print(losses)  # The loss decreased every iteration over the training data!

[696.5410079956055, 690.024829864502, 683.5933752059937, 677.2429785728455, 670.9677383899689, 664.7610003948212, 658.6181926727295, 652.5369682312012, 646.5131125450134, 640.5408644676208, 634.617128610611, 628.7403244972229, 622.905074596405, 617.1078517436981, 611.3453333377838, 605.6135358810425, 599.9097530841827, 594.2266871929169, 588.5661828517914, 582.9253082275391, 577.3002414703369, 571.6864535808563, 566.0819356441498, 560.4866931438446, 554.8949990272522, 549.3098056316376, 543.7241456508636, 538.1383891105652, 532.5465577840805, 526.9513831138611, 521.3494460582733, 515.7381641864777, 510.11786460876465, 504.48786973953247, 498.8487138748169, 493.19515311717987, 487.5297852754593, 481.8497380018234, 476.15810215473175, 470.4516968727112, 464.7282450199127, 458.98999309539795, 453.23730194568634, 447.46762907505035, 441.68490517139435, 435.8833876848221, 430.0674341917038, 424.2374038696289, 418.395822763443, 412.5411710739136, 406.6766719222069, 400.80423736572266, 394.92

In [14]:
# list out keys and values separately
key_list = list(word_to_ix.keys())
val_list = list(word_to_ix.values())

In [15]:
def similarity_cbow(word_1, word_2):
    
    # test word similarity
    print(word_1)
    print(word_2)
    w1_id = torch.tensor(word_to_ix[word_1], dtype=torch.long)
    w2_id = torch.tensor(word_to_ix[word_2], dtype=torch.long)
    w1_id = w1_id.to(device)
    w2_id = w2_id.to(device)
    
    word_1_vec = model.get_word_vector(w1_id)
    word_2_vec = model.get_word_vector(w2_id)
    
    # The norm of a vector (1D-matrix) is the square root of the sum of all the squared values within the vector.
    print(math.sqrt(torch.square(word_1_vec).sum()))    
    print(torch.linalg.norm(word_1_vec))
    print(torch.linalg.norm(word_2_vec))
    print(word_1_vec.dot(word_2_vec))
    
    word_distance = torch.linalg.norm(word_1_vec - word_2_vec)
    print("Distance between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_distance))
    word_similarity = (word_1_vec.dot(word_2_vec) / (torch.linalg.norm(word_1_vec) * torch.linalg.norm(word_2_vec)))
    print("Similarity between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_similarity))


In [16]:
similarity_cbow("neural", "network")

neural
network
3.9110193052739572
tensor(3.9110, device='cuda:0', grad_fn=<CopyBackwards>)
tensor(3.1197, device='cuda:0', grad_fn=<CopyBackwards>)
tensor(4.2350, device='cuda:0', grad_fn=<DotBackward>)
Distance between 'neural' & 'network' : 4.0692
Similarity between 'neural' & 'network' : 0.3471


In [17]:
def predict_middle_word(prev_words, post_words):
    prev_words = prev_words.split()
    post_words = post_words.split()

    input_words= make_context_vector(prev_words + post_words, word_to_ix)
    input_words = input_words.to(device)
    output = model(input_words)
    out_ind = output.argmax(1)
#     print(word_to_ix)
#     out_word = word_to_ix.itos[out_ind.item()]
    out_word = key_list[val_list.index(out_ind.item())]
    print(out_word)

In [18]:
predict_middle_word("a recurrent", "network is")
predict_middle_word("LSTM is", "recurrent neural")

neural
a


### Now that you saw how to create the CBOW model (word2vec), you should work on doing the "opposite" model, Skip-Gram ###

Skip-gram as you saw on the lectures, reverses the problem so you need to predict through the "fake task" the context of the input

# Lab assignment below

In [19]:
class Skipgram(nn.Module):
    def __init__(self, vocab_size, embed_dim, context,target_size, hidden_size):
        super(Skipgram, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(embed_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, vocab_size*target_size),
            nn.LogSoftmax(dim = 1)
        )
        self.target_size=target_size
        
    def forward(self, inputs):
      #  print("0", inputs.shape)
#         print(inputs)
        out = self.embedding(inputs)
     #   print("1", out.shape)
        #print(out.shape)
        #out = out.view(1, -1)
#         print(out.shape)
        out = self.linear(out)
      #  print("2", out.shape)
        out = out.view(self.target_size, -1)
      #  print("3", out.shape)
        return out
    
    # This is what we are actually interested on
    def get_word_vector(self, word):
        out = self.embedding(word)
        return out

In [20]:
def train(model, epochs, data, optimizer, loss_fn):
    model.train()
    losses = []
    for epoch in range(epochs):
        total_loss = 0
        for target, context in data:

            # Prepare inputs and targets 
            context_idxs = make_context_vector([context], word_to_ix)
            context_idxs = context_idxs.to(device)

            target_id = make_context_vector(target, word_to_ix)
            target_id = target_id.to(device)
            
            # Do not accumulate 
            model.zero_grad()

            # Step 3. Run the forward pass
            log_probs = model(context_idxs)
    #         break

            # Step 4. Compute your loss function.
            loss = loss_fn(log_probs, target_id)

    #       loss = loss_function(log_probs, torch.tensor([word_to_ix[context]], dtype=torch.long))

            # Step 5. Do the backward pass and update the gradient
            loss.backward()
            optimizer.step()

            # Get the Python number from a 1-element Tensor by calling tensor.item()
            total_loss += loss.item()
        losses.append(total_loss)
    return losses
    

In [21]:
VOCAB_SIZE = len(vocab)
EMBEDD_DIM = 106
CONTEXT_SIZE = 1
HIDDEN_SIZE = 512 # 256
TARGET_SIZE = 4

loss_function = nn.NLLLoss() # Because we are using 
model = Skipgram(vocab_size, EMBEDD_DIM, CONTEXT_SIZE, TARGET_SIZE, HIDDEN_SIZE)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

losses = train(model, 100, data, optimizer, loss_function)
model.eval()

print(losses)  # The loss decreased every iteration over the training data!

[899.4872632026672, 895.7377047538757, 892.0125298500061, 888.3110189437866, 884.6314563751221, 880.9740219116211, 877.3364405632019, 873.7178478240967, 870.1160531044006, 866.5306372642517, 862.9610242843628, 859.4066505432129, 855.8679509162903, 852.3429064750671, 848.8295178413391, 845.3266372680664, 841.8338770866394, 838.3502655029297, 834.874819278717, 831.4069576263428, 827.9452252388, 824.4900088310242, 821.0399055480957, 817.5947027206421, 814.1534395217896, 810.7163238525391, 807.2818970680237, 803.8498058319092, 800.4193277359009, 796.989794254303, 793.5617184638977, 790.1352472305298, 786.7105660438538, 783.2864217758179, 779.8612594604492, 776.4359555244446, 773.010055065155, 769.5830039978027, 766.153281211853, 762.7231545448303, 759.2903356552124, 755.8557887077332, 752.4176535606384, 748.9777035713196, 745.5344352722168, 742.0883955955505, 738.6394891738892, 735.1867532730103, 731.7312579154968, 728.2709567546844, 724.8076317310333, 721.340815782547, 717.8692309856415, 

In [22]:
#import matplotlib.pyplot as plt
#plt.figure()
#plt.plot(losses)

In [23]:
# list out keys and values separately
key_list = list(word_to_ix.keys())
val_list = list(word_to_ix.values())
print(key_list)
print(val_list)

['paper,', 'in', 'been', 'phone', 'limited', 'Unlike', 'models', 'sequences.', 'modeling,', 'the', 'models.', 'cyclic', 'present', 'such', 'more', 'We', 'which', 'successfully', 'at', 'tasks.', 'feedforward', 'a', 'prediction', 'exploding', 'sized', 'speech', 'However,', 'effective', 'networks,', 'making', 'handwriting', 'labeling', 'to', '(LSTM)', 'vanishing', '(RNN)', 'gradient', 'deep', 'we', 'for', 'train', 'use', 'powerful', 'connections', 'problems', 'state', 'RNN', 'LSTM,', 'them', 'sequence', 'art', 'network', 'recurrent', 'acoustic', 'parameters', 'vocabulary', 'Memory', 'conventional', 'scale', 'numbers', 'is', 'based', 'this', 'Long', 'language', 'has', 'recognition,', 'tasks,', 'recognition', 'used', 'novel', 'of', 'as', 'DNN', 'frames.', 'architectures', 'small', 'contrast', 'They', 'show', 'architecture', 'RNNs', 'make', 'quickly', 'that', 'modeling', 'model', 'and', 'RNNs.', 'Short-Term', 'converge', 'designed', 'compare', 'phonetic', 'relatively', 'large', 'configuratio

Reverse cbow and use the middle word to predict the 2 words adjacent to the middle word.

In [24]:
"""
def predict_middle_word(prev_words, post_words):
    prev_words = prev_words.split()
    post_words = post_words.split()

    input_words= make_context_vector(prev_words + post_words, word_to_ix)
    input_words = input_words.to(device)
    output = model(input_words)
    out_ind = output.argmax(1)
#     print(word_to_ix)
#     out_word = word_to_ix.itos[out_ind.item()]
    out_word = key_list[val_list.index(out_ind.item())]
    print(out_word)

"""

# Reverse cbow and use the middle word to predict the 2 words adjacent to the middle word.
def predict_words(middle_word):
    context_words = []
    
    print("Word context:", middle_word)
    input_words = make_context_vector([middle_word], word_to_ix)
    input_words = input_words.to(device)
    #output = model.get_word_vector(input_words)
    output = model(input_words)
    prediction = output.argmax(1)
   # pred = torch.topk(output, 4) # return the most likely elements/words. https://pytorch.org/docs/stable/generated/torch.topk.html 
    #print("predictions", prediction)
    print("oi", prediction)  
    
    #for word in prediction[1][0]:
    for word in prediction:
        print("word tensor:", word)
        context_words.append(key_list[word])
       # print("test", context_words)
    context_words.insert(int(len(context_words)/2), middle_word)
    print("\n", context_words)



In [25]:
predict_words("neural")

Word context: neural
oi tensor([ 9, 20, 28,  9], device='cuda:0')
word tensor: tensor(9, device='cuda:0')
word tensor: tensor(20, device='cuda:0')
word tensor: tensor(28, device='cuda:0')
word tensor: tensor(9, device='cuda:0')

 ['the', 'feedforward', 'neural', 'networks,', 'the']


In [26]:
"""   
def predict_topk(middle_word):
    print("Word context:", middle_word)
    input_words = make_context_vector([middle_word], word_to_ix)
    input_words = input_words.to(device)
    output = model.get_word_vector(input_words)
    #out_index = output.argmax(1)
    
    prediction = torch.topk(output, 4) # return the most likely elements/words. https://pytorch.org/docs/stable/generated/torch.topk.html 
    print("predictions", prediction)
  
    context_words=[]
    for word in prediction[1][0]:
        print("word tensor", word)
        context_words.append(key_list[word])
       # print("test", context_words)
    context_words.insert(int(len(context_words)/2), middle_word)
    print(context_words)
"""


'   \ndef predict_topk(middle_word):\n    print("Word context:", middle_word)\n    input_words = make_context_vector([middle_word], word_to_ix)\n    input_words = input_words.to(device)\n    output = model.get_word_vector(input_words)\n    #out_index = output.argmax(1)\n    \n    prediction = torch.topk(output, 4) # return the most likely elements/words. https://pytorch.org/docs/stable/generated/torch.topk.html \n    print("predictions", prediction)\n  \n    context_words=[]\n    for word in prediction[1][0]:\n        print("word tensor", word)\n        context_words.append(key_list[word])\n       # print("test", context_words)\n    context_words.insert(int(len(context_words)/2), middle_word)\n    print(context_words)\n'

In [27]:
#predict_topk("neural")

In [28]:
#similarity_cbow("neural", "network")